In [4]:
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

ImportError: cannot import name 'KerasLazyLoader' from 'tensorflow.python.util.lazy_loader' (C:\Users\erand\anaconda3\envs\gpu_1\lib\site-packages\tensorflow\python\util\lazy_loader.py)

In [ ]:
filename = "mental_H.txt"
raw_text = open(filename, "r", encoding="utf-8", errors="ignore").read()
raw_text = raw_text.lower()

In [ ]:
import torch
print("Is CUDA available: ", torch.cuda.is_available())
print("Number of GPUs: ", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU Name: ", torch.cuda.get_device_name(0))


In [3]:
import torch

print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Number of GPU:  1
GPU Name:  NVIDIA GeForce RTX 3050 Ti Laptop GPU
Using device: cuda


In [11]:
raw_text

'mental health\nmental health is a state of mental well-being that enables people to cope with the stresses of life, realize their abilities, learn well and work well, and contribute to their community. it has intrinsic and instrumental value and is integral to our well-being. \nat any one time, a diverse set of individual, family, community and structural factors may combine to protect or undermine mental health. although most people are resilient, people who are exposed to adverse circumstances  including poverty, violence, disability and inequality  are at higher risk of developing a mental health condition.\nmany mental health conditions can be effectively treated at relatively low cost, yet health systems remain significantly under-resourced and treatment gaps are wide all over the world. mental health care is often poor in quality when delivered. people with mental health conditions often also experience stigma, discrimination and human rights violations.\n\nkey facts\n* affordab

In [12]:
chars = sorted(list(set(raw_text)))

In [13]:
chars

['\n',
 ' ',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '?',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|']

In [14]:
len(chars)

59

In [15]:
len(raw_text)

122417

In [16]:
char_to_int = dict((c,i) for i,c in enumerate(chars))

In [17]:
char_to_int

{'\n': 0,
 ' ': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 '*': 9,
 '+': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '=': 27,
 '?': 28,
 '[': 29,
 ']': 30,
 '_': 31,
 'a': 32,
 'b': 33,
 'c': 34,
 'd': 35,
 'e': 36,
 'f': 37,
 'g': 38,
 'h': 39,
 'i': 40,
 'j': 41,
 'k': 42,
 'l': 43,
 'm': 44,
 'n': 45,
 'o': 46,
 'p': 47,
 'q': 48,
 'r': 49,
 's': 50,
 't': 51,
 'u': 52,
 'v': 53,
 'w': 54,
 'x': 55,
 'y': 56,
 'z': 57,
 '|': 58}

In [18]:
n_chars = len(raw_text)
n_vocab = len(chars)

In [19]:
seq_len = 100
dataX = []
dataY = []
for i in range(0,n_chars-seq_len,1):
    seq_in = raw_text[i:i+seq_len] #0-99, train 99 values
    seq_out = raw_text[i+seq_len] #100, for 100th
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total patterns : ", n_patterns)

Total patterns :  122317


In [20]:
len(dataX[0])

100

In [21]:
dataY[20]

57

In [22]:
X = numpy.reshape(dataX , (n_patterns,seq_len,1)) #(samples, timesans, features)
X = X/float(n_vocab) #0-1
y = to_categorical(dataY)

In [23]:
X.shape

(122317, 100, 1)

In [24]:
#model = Sequential()
#model.add(LSTM(256, input_shape= (X.shape[1],X.shape[2]))) #100,1
#model.add(Dropout(0.2)) #20%
#model.add(Dense(y.shape[1], activation="softmax")) #y.shape[1] = num of voc size

In [25]:
from tensorflow.keras.layers import BatchNormalization

model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(256))  # Second LSTM layer
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation="softmax"))

NotImplementedError: Cannot convert a symbolic Tensor (lstm/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [26]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [27]:
filepath = "weights-improvement-{epoch:01d}-{loss:.4f}.hdf5" #store path and type
checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=True)
#verbose = display the description when it was save
callbacks_list = [checkpoint]

In [28]:
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [29]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20


ValueError: in user code:

    C:\Users\erand\anaconda3\envs\gpu1\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\erand\anaconda3\envs\gpu1\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\erand\anaconda3\envs\gpu1\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\erand\anaconda3\envs\gpu1\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\erand\anaconda3\envs\gpu1\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\erand\anaconda3\envs\gpu1\lib\site-packages\t

In [ ]:
f_name = "weights-improvement-20-1.9504.hdf5"
model.load_weights(f_name)
model.compile(loss="categorical_crossentropy")
model.compile(loss="categorical_crossentropy",optimizer="adam")

In [ ]:
int_to_char = dict((i,c) for i,c in enumerate(chars))

In [ ]:
int_to_char

In [ ]:
import sys

In [ ]:
start = numpy.random.randint(0,len(dataX))
pattern = dataX[start]
print("seed")
print("\"", "".join([int_to_char[value] for value in pattern]),"\"")

for i in range(1000):
    x = numpy.reshape(pattern,(1,len(pattern),1))
    x = x/float(n_vocab)
    prediction = model.predict(x,verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]